In [4]:
import pandas as pd
from graphviz import Digraph
import numpy as np
def find_allxiaji_proxies(df, proxyName, result=[]):
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['proxyName'] == proxyName]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['username'])
        # Recursively call the function with the new proxyName
        find_allxiaji_proxies(df, row['username'], result)
    return result
def find_allshangji_proxies(df, username, result=[]):
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['username'] == username]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['proxyName'])
        # Recursively call the function with the new proxyName
        find_allshangji_proxies(df, row['proxyName'], result)
    return result
def df_to_dict(df):
    """
    将三列数据的DataFrame转换为字典
    """
    result = df.set_index(df.columns[0]).to_dict('index')
    return result
df = pd.read_excel(r"D:\项目\鄂州\所需数据\会员信息代理关系.xlsx", sheet_name=0)
df_yh = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='用户信息',
                      dtype={'身份证':str})
hy_jz= pd.read_excel(r"D:\项目\鄂州\所需数据\会员管理_会员收款介质.xlsx", sheet_name=0,dtype={'PAY_ACCOUNT':str})

hy_ka1=hy_jz[['USER_NAME','PAY_ACCOUNT']]
hy_ka=hy_ka1.rename(columns={'USER_NAME':'查询账号','PAY_ACCOUNT':'账户'})
result_yh_dt = df_to_dict(df_yh)
agent_find = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='要查找的代理')
agent_l=list(agent_find['查询账号'])
l_all_ka=[]
kas=[]
for i in agent_l:
    result_xia = find_allxiaji_proxies(df,i, result=[])
    #print(result)
    result_shang = find_allshangji_proxies(df,i, result=[])
    data1=df[df['username'].isin(result_xia)]
    data2=df[df['proxyName'].isin(result_shang)]
    data=pd.concat([data1,data2])
    l_all_ka.append(data)
    photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
    photo.graph_attr['rankdir'] = 'LR' 
    nodes = {}  # 存储所有节点名称的字典

    for index, row in data.iterrows():
            mother = row['proxyName']  # 母公司名称

            if mother in result_yh_dt :
                    #mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['收款账号'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['姓名'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    #print(mother_name)
            else:
                    mother_name = str(mother)
            child = row['username']
            if child in result_yh_dt :
                #child_name = str(child)+ '\n'+str(result_yh_dt[child]['收款账号'])+'\n'+str(result_yh_dt[child]['身份证'])
                    child_name = str(child)+ '\n'+str(result_yh_dt[child]['姓名'])+'\n'+str(result_yh_dt[child]['身份证'])
            else:
                    child_name = str(child)
    #         else:
    #             s=s+row['用户名 姓名 登录IP']+'\n'
            if mother_name not in nodes.keys():  # 如果没有画过该节点
                photo.node(name=str(mother_name), fontname="Microsoft YaHei")  # 画母公司节点

            if child_name not in nodes.keys():
            # print(str(index))
                photo.node(name=str(child_name), fontname="Microsoft YaHei")  # 画子公司节点

                photo.edge(str(child_name),str(mother_name))
            # print(nodes.items())
            # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
    photo.render(filename='%s' % (i),outfile= None,directory=r'D:\项目\鄂州\代理\代理前10\{}'.format(i))
    u1=list(data['username'])
    p1=list(data['proxyName'])
    l=list(set(u1+p1))
    data_info=pd.DataFrame({'查询账号':l})
    data_info['所在代理']=i


    
    data_ka=data_info.merge(hy_ka,on='查询账号',how='left')
    agent_info=data_ka.merge(df_yh,on='查询账号',how='left')
    agent_info['所在代理']=i
    kas.append(agent_info)
    with pd.ExcelWriter(r'D:\项目\鄂州\代理\代理前10\{}\{}.xlsx'.format(i, i)) as writer:
            data.to_excel(writer, sheet_name="代理表",index=False)  
            agent_info.to_excel(writer, sheet_name="代理账号信息",index=False)
    

agent_kas=pd.concat(l_all_ka)
agent_kas.to_excel(r'D:\项目\鄂州\代理\代理前10\银行卡信息.xlsx',index=False)
ka=pd.concat(kas)
ka=ka.drop_duplicates()
ka.to_excel(r'D:\项目\鄂州\代理\代理前10\代理银行卡信息.xlsx',index=False)

In [18]:
agent_kas=pd.concat(l_all_ka)
agent_kas.to_excel(r'D:\项目\鄂州\代理\银行卡信息.xlsx',index=False)

In [11]:
#只画图
def find_all_proxies(df, proxyName, result=[]):
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['proxyName'] == proxyName]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['username'])
        # Recursively call the function with the new proxyName
        find_all_proxies(df, row['username'], result)
    return result
def df_to_dict(df):
    """
    将三列数据的DataFrame转换为字典
    """
    result = df.set_index(df.columns[0]).to_dict('index')
    return result


# 测试

# Example usage
df = pd.read_excel(r"D:\项目\鄂州\所需数据\会员信息代理关系.xlsx", sheet_name=1)
df_yh = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='用户信息',
                      dtype={'身份证':str})
result_yh_dt = df_to_dict(df_yh)
result = find_all_proxies(df, 'al2162645')
#print(result)

data=df[df['username'].isin(result)]
i='al2162645'
photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
photo.graph_attr['rankdir'] = 'LR' 
nodes = {}  # 存储所有节点名称的字典

for index, row in data.iterrows():
        mother = row['proxyName']  # 母公司名称
           
        if mother in result_yh_dt :
                mother_name = str(mother)+ '\n'+result_yh_dt[mother]['收款账号']+'\n'+str(result_yh_dt[mother]['身份证'])
                #print(mother_name)
        else:
                mother_name = str(mother)
        child = row['username']
        if child in result_yh_dt :
            child_name = str(child)+ '\n'+result_yh_dt[child]['收款账号']+'\n'+str(result_yh_dt[child]['身份证'])
        else:
                child_name = str(child)
#         else:
#             s=s+row['用户名 姓名 登录IP']+'\n'
        if mother_name not in nodes.keys():  # 如果没有画过该节点
            photo.node(name=str(mother_name), fontname="Microsoft YaHei")  # 画母公司节点

        if child_name not in nodes.keys():
        # print(str(index))
            photo.node(name=str(child_name), fontname="Microsoft YaHei")  # 画子公司节点

            photo.edge(str(mother_name), str(child_name))
        # print(nodes.items())
        # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
photo.render(filename='%s' % (i),outfile= None,directory=r'C:\Users\23934\Desktop\雅安\代理数据分析\代理图1\{}'.format(i))
#xx.to_excel(r'C:\Users\23934\Desktop\项目\磐石\代理\代理数据\代理图\{}\{}.xlsx'.format(i, i))

'C:\\Users\\23934\\Desktop\\雅安\\代理数据分析\\代理图1\\al2162645\\al2162645.pdf'

In [25]:
#直属上下两级代理关系图及账号信息表
import pandas as pd
from graphviz import Digraph
def df_to_dict(df):
    """
    将三列数据的DataFrame转换为字典
    """
    result = df.set_index(df.columns[0]).to_dict('index')
    return result
df = pd.read_excel(r"D:\项目\鄂州\所需数据\会员信息代理关系.xlsx", sheet_name=0)
df_yh = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='用户信息',
                      dtype={'身份证':str})
hy_jz= pd.read_excel(r"D:\项目\鄂州\所需数据\会员管理_会员收款介质.xlsx", sheet_name=0,dtype={'PAY_ACCOUNT':str})

hy_ka1=hy_jz[['USER_NAME','PAY_ACCOUNT']]
hy_ka=hy_ka1.rename(columns={'USER_NAME':'查询账号','PAY_ACCOUNT':'账户'})
result_yh_dt = df_to_dict(df_yh)
agent_find = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='要查找的代理')
agent_l=list(agent_find['查询账号'])
l_all_ka=[]
kas=[]
l=[]
for i in agent_l:
    subordinates=[]
    superiors=[]
    direct_subordinates = df[df['proxyName'] ==i]['username'].tolist()
    print("一级下属:", direct_subordinates)
    subordinates.append(direct_subordinates)
    # 找到直属下级的下级
    subordinates_of_subordinates = df[df['proxyName'].isin(direct_subordinates)]['username'].tolist()
    subordinates.append(subordinates_of_subordinates)
    print("二级下属:", subordinates_of_subordinates)
    direct_superiors = df[df['username'] == i]['proxyName'].tolist()
    superiors.append(direct_superiors)
    superiors_of_superiors = df[df['username'].isin(direct_superiors)]['proxyName'].tolist()
    superiors.append(superiors_of_superiors)

    subordinates = [x for sublst in subordinates for x in sublst]
    superiors=[x for sublst in superiors for x in sublst]
    l.extend([[i], subordinates, superiors])
    new_lst = [x for sublst in l for x in sublst]
    new_lst_qc=list(set(new_lst))
    data1=df[df['username'].isin(subordinates)]
    data2=df[df['proxyName'].isin(superiors)]
    data=pd.concat([data1,data2])
    l_all_ka.append(data)

    photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
    photo.graph_attr['rankdir'] = 'LR' 
    nodes = {}  # 存储所有节点名称的字典

    for index, row in data.iterrows():
            mother = row['proxyName']  # 母公司名称

            if mother in result_yh_dt :
                    #mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['收款账号'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['姓名'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    #print(mother_name)
            else:
                    mother_name = str(mother)
            child = row['username']
            if child in result_yh_dt :
                #child_name = str(child)+ '\n'+str(result_yh_dt[child]['收款账号'])+'\n'+str(result_yh_dt[child]['身份证'])
                    child_name = str(child)+ '\n'+str(result_yh_dt[child]['姓名'])+'\n'+str(result_yh_dt[child]['身份证'])
            else:
                    child_name = str(child)
    #         else:
    #             s=s+row['用户名 姓名 登录IP']+'\n'
            if mother_name not in nodes.keys():  # 如果没有画过该节点
                photo.node(name=str(mother_name), fontname="Microsoft YaHei")  # 画母公司节点

            if child_name not in nodes.keys():
            # print(str(index))
                photo.node(name=str(child_name), fontname="Microsoft YaHei")  # 画子公司节点

                photo.edge(str(child_name),str(mother_name))
            # print(nodes.items())
            # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
    photo.render(filename='%s' % (i),outfile= None,directory=r'D:\项目\鄂州\代理\代理前10上下级2\{}'.format(i))


    data_infos = {'一级下属':direct_subordinates, '二级下属': subordinates_of_subordinates,'直接上级':direct_superiors,'二级上级':superiors_of_superiors}
    out= pd.DataFrame(pd.DataFrame.from_dict(data_infos, orient='index').values.T, columns=list(data_infos.keys()))
    data_info=pd.DataFrame({'查询账号':new_lst_qc})
    data_ka=data_info.merge(hy_ka,on='查询账号',how='left')
    agent_info=data_ka.merge(df_yh,on='查询账号',how='left')
    kas.append(agent_info)
    with pd.ExcelWriter(r'D:\项目\鄂州\代理\代理前10上下级2\{}\{}.xlsx'.format(i, i)) as writer:
            data.to_excel(writer, sheet_name="代理关系",index=False)  
            agent_info.to_excel(writer, sheet_name="代理账号信息",index=False)
            out.to_excel(writer, sheet_name="代理级别",index=False)
agent_kas=pd.concat(l_all_ka)
#agent_kas.to_excel(r'D:\项目\鄂州\代理\代理前10上下级2\银行卡信息.xlsx',index=False)
ka=pd.concat(kas)
ka=ka.drop_duplicates()
#ka.to_excel(r'D:\项目\鄂州\代理\代理前10上下级2\代理银行卡信息.xlsx',index=False)
with pd.ExcelWriter(r'D:\项目\鄂州\代理\代理前10上下级2\代理银行卡信息.xlsx') as writer:
            agent_kas.to_excel(writer, sheet_name="代理关系",index=False)  
            ka.to_excel(writer, sheet_name="代理账号信息",index=False)


In [5]:
#230608所有下线信息
import pandas as pd
from graphviz import Digraph
import numpy as np
def find_allxiaji_proxies(df, proxyName, result=[]):
    
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['proxyName'] == proxyName]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['username'])
        # Recursively call the function with the new proxyName
        find_allxiaji_proxies(df, row['username'], result)
    return result

def df_to_dict(df):
    """
    将三列数据的DataFrame转换为字典
    """
    result = df.set_index(df.columns[0]).to_dict('index')
    return result
df = pd.read_excel(r"D:\项目\鄂州\所需数据\会员信息代理关系.xlsx", sheet_name=0)
df_yh = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='用户信息',
                      dtype={'身份证':str})
hy_jz= pd.read_excel(r"D:\项目\鄂州\所需数据\会员管理_会员收款介质.xlsx", sheet_name=0,dtype={'PAY_ACCOUNT':str})

hy_ka1=hy_jz[['USER_NAME','PAY_ACCOUNT']]
hy_ka=hy_ka1.rename(columns={'USER_NAME':'查询账号','PAY_ACCOUNT':'账户'})
result_yh_dt = df_to_dict(df_yh)
agent_find = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='要查找的代理')
agent_l=list(agent_find['查询账号'])
l_all_ka=[]
kas=[]
for i in agent_l:
    result_xia = find_allxiaji_proxies(df,i,result=[])
    #print(result)

    data=df[df['username'].isin(result_xia)]

    l_all_ka.append(data)
    photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
    photo.graph_attr['rankdir'] = 'LR' 
    nodes = {}  # 存储所有节点名称的字典

    for index, row in data.iterrows():
            mother = row['proxyName']  # 母公司名称

            if mother in result_yh_dt :
                    #mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['收款账号'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['姓名'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    #print(mother_name)
            else:
                    mother_name = str(mother)
            child = row['username']
            if child in result_yh_dt :
                #child_name = str(child)+ '\n'+str(result_yh_dt[child]['收款账号'])+'\n'+str(result_yh_dt[child]['身份证'])
                    child_name = str(child)+ '\n'+str(result_yh_dt[child]['姓名'])+'\n'+str(result_yh_dt[child]['身份证'])
            else:
                    child_name = str(child)
    #         else:
    #             s=s+row['用户名 姓名 登录IP']+'\n'
            if mother_name not in nodes.keys():  # 如果没有画过该节点
                photo.node(name=str(mother_name), fontname="Microsoft YaHei")  # 画母公司节点

            if child_name not in nodes.keys():
            # print(str(index))
                photo.node(name=str(child_name), fontname="Microsoft YaHei")  # 画子公司节点

                photo.edge(str(child_name),str(mother_name))
            # print(nodes.items())
            # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
    photo.render(filename='%s' % (i),outfile= None,directory=r'D:\项目\鄂州\代理\代理下线1\{}'.format(i))
    u1=list(data['username'])
    p1=list(data['proxyName'])
    l=list(set(u1+p1))
    data_info=pd.DataFrame({'查询账号':l})
    data_info['所在代理']=i


    
    data_ka=data_info.merge(hy_ka,on='查询账号',how='left')
    agent_info=data_ka.merge(df_yh,on='查询账号',how='left')
    agent_info['所在代理']=i
    kas.append(agent_info)
#     with pd.ExcelWriter(r'D:\项目\鄂州\代理前10\{}\{}.xlsx'.format(i, i)) as writer:
#             data.to_excel(writer, sheet_name="代理表",index=False)  
#             agent_info.to_excel(writer, sheet_name="代理账号信息",index=False)
    #data.to_excel(r'C:\Users\23934\Desktop\\代理图1\{}\{}.xlsx'.format(i, i),index=False)
#l_all_ka.to_excel(r'D:\项目\鄂州\代理图\银行卡信息.xlsx',index=False)
agent_kas=pd.concat(l_all_ka)
agent_kas=agent_kas.drop_duplicates()
agent_kas.to_excel(r'D:\项目\鄂州\代理\代理下线1\银行卡信息.xlsx',index=False)
ka=pd.concat(kas)
ka=ka.drop_duplicates()
ka.to_excel(r'D:\项目\鄂州\代理\代理下线1\代理银行卡信息.xlsx',index=False)

In [5]:
#230608所有下线信息
import pandas as pd
from graphviz import Digraph
import numpy as np
def find_allxiaji_proxies(df, proxyName, result=[]):
    
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['proxyName'] == proxyName]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['username'])
        # Recursively call the function with the new proxyName
        find_allxiaji_proxies(df, row['username'], result)
    return result

def df_to_dict(df):
    """
    将三列数据的DataFrame转换为字典
    """
    result = df.set_index(df.columns[0]).to_dict('index')
    return result
df = pd.read_excel(r"D:\项目\鄂州\所需数据\会员信息代理关系.xlsx", sheet_name=0)
df_yh = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='用户信息713',
                      dtype={'身份证':str})
hy_jz= pd.read_excel(r"D:\项目\鄂州\所需数据\会员管理_会员收款介质.xlsx", sheet_name=0,dtype={'PAY_ACCOUNT':str})

hy_ka1=hy_jz[['USER_NAME','PAY_ACCOUNT']]
hy_ka=hy_ka1.rename(columns={'USER_NAME':'查询账号','PAY_ACCOUNT':'账户'})
result_yh_dt = df_to_dict(df_yh)
agent_find = pd.read_excel(r"D:\项目\鄂州\中转文件\前10代理.xlsx", sheet_name='代理24')
agent_l=list(agent_find['查询账号'])
l_all_ka=[]
kas=[]
for i in agent_l:
    result_xia = find_allxiaji_proxies(df,i,result=[])
    #print(result)

    data=df[df['username'].isin(result_xia)]

    l_all_ka.append(data)
    photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
    photo.graph_attr['rankdir'] = 'LR' 
    nodes = {}  # 存储所有节点名称的字典

    for index, row in data.iterrows():
            mother = row['proxyName']  # 母公司名称

            if mother in result_yh_dt :
                    #mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['收款账号'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['姓名'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    #print(mother_name)
            else:
                    mother_name = str(mother)
            child = row['username']
            if child in result_yh_dt :
                #child_name = str(child)+ '\n'+str(result_yh_dt[child]['收款账号'])+'\n'+str(result_yh_dt[child]['身份证'])
                    child_name = str(child)+ '\n'+str(result_yh_dt[child]['姓名'])+'\n'+str(result_yh_dt[child]['身份证'])
            else:
                    child_name = str(child)
    #         else:
    #             s=s+row['用户名 姓名 登录IP']+'\n'
            if mother_name not in nodes.keys():  # 如果没有画过该节点
                photo.node(name=str(mother_name),fontname='KaiTi')  # 画母公司节点

            if child_name not in nodes.keys():
            # print(str(index))
                photo.node(name=str(child_name),fontname='KaiTi')  # 画子公司节点

                photo.edge(str(child_name),str(mother_name))
            # print(nodes.items())
            # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
    photo.render(filename='%s' % (i),outfile= None,directory=r'D:\项目\鄂州\代理\代理下线1\{}'.format(i))
    u1=list(data['username'])
    p1=list(data['proxyName'])
    l=list(set(u1+p1))
    data_info=pd.DataFrame({'查询账号':l})
    data_info['所在代理']=i


    
    data_ka=data_info.merge(hy_ka,on='查询账号',how='left')
    agent_info=data_ka.merge(df_yh,on='查询账号',how='left')
    agent_info['所在代理']=i
    kas.append(agent_info)
#     with pd.ExcelWriter(r'D:\项目\鄂州\代理前10\{}\{}.xlsx'.format(i, i)) as writer:
#             data.to_excel(writer, sheet_name="代理表",index=False)  
#             agent_info.to_excel(writer, sheet_name="代理账号信息",index=False)
    #data.to_excel(r'C:\Users\23934\Desktop\\代理图1\{}\{}.xlsx'.format(i, i),index=False)
#l_all_ka.to_excel(r'D:\项目\鄂州\代理图\银行卡信息.xlsx',index=False)
agent_kas=pd.concat(l_all_ka)
agent_kas=agent_kas.drop_duplicates()
agent_kas.to_excel(r'D:\项目\鄂州\代理\代理下线1\银行卡信息71311.xlsx',index=False)
ka=pd.concat(kas)
ka=ka.drop_duplicates()
ka.to_excel(r'D:\项目\鄂州\代理\代理下线1\代理银行卡信息71311.xlsx',index=False)